In [30]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [78]:
response = requests.get("https://www.lesoir.be/18/sections/le-direct")
soup = bs(response.content, "html.parser")


#### Create list with urls of first page of news articles

In [79]:
urls = []
base_url = "https://www.lesoir.be"
links = soup.select("h3 > a")
for link in links:
    url = link.get('href')
    dict = {'source_url': url if '//' in url else base_url + url }
    urls.append(dict)   
print(urls)


[{'source_url': 'https://www.lesoir.be/529030/article/2023-08-02/vetements-pour-enfants-dangereux-lavertissement-du-spf-economie'}, {'source_url': 'https://www.lesoir.be/529026/article/2023-08-02/vol-avec-violences-la-police-lance-un-avis-de-recherche-photo'}, {'source_url': 'https://www.lesoir.be/529025/article/2023-08-02/pres-dune-tonne-de-cocaine-saisie-et-cinq-arrestations-au-port-danvers'}, {'source_url': 'https://www.lesoir.be/529024/article/2023-08-02/explosion-du-taux-de-criminalite-au-japon-linfaillible-securite-publique-en'}, {'source_url': 'https://www.lesoir.be/529022/article/2023-08-02/averses-intenses-rafales-de-vent-orage-lirm-lance-deux-alertes-jaunes'}, {'source_url': 'https://www.lesoir.be/529018/article/2023-08-02/guerre-en-ukraine-les-frappes-russes-sur-le-danube-ont-endommage-40000-tonnes-de'}, {'source_url': 'https://www.lesoir.be/529012/article/2023-08-02/intemperies-risque-de-tornades-et-de-violentes-rafales-de-vent'}, {'source_url': 'https://www.lesoir.be/52901

In [80]:
df = pd.DataFrame(urls)
df.head(20)

,source_url
0,https://www.lesoir.be/529030/article/2023-08-0...
1,https://www.lesoir.be/529026/article/2023-08-0...
2,https://www.lesoir.be/529025/article/2023-08-0...
3,https://www.lesoir.be/529024/article/2023-08-0...
4,https://www.lesoir.be/529022/article/2023-08-0...
5,https://www.lesoir.be/529018/article/2023-08-0...
6,https://www.lesoir.be/529012/article/2023-08-0...
7,https://www.lesoir.be/529011/article/2023-08-0...
8,https://sosoir.lesoir.be/le-premier-festival-s...
9,https://www.lesoir.be/529001/article/2023-08-0...


In [83]:
url_one = df['source_url'][0]
url_one

'https://www.lesoir.be/529030/article/2023-08-02/vetements-pour-enfants-dangereux-lavertissement-du-spf-economie'

In [84]:
response_2 = requests.get(url_one)
soup2 = bs(response_2.content, 'html.parser')
paragraphs = soup2.find_all('p')
print(len(paragraphs))
for paragrapg in paragraphs:
    print(paragrapg)


11
<p><span class="drop-cap r-lettrine">P</span>lus de trois quarts des vêtements pour enfants analysés lors d’une campagne de contrôle n’étaient pas conformes aux exigences de sécurité, rapporte le SPF Economie lundi. Sur les 40 vêtements d’enfants jusqu’à 14 ans soumis à un contrôle approfondi, 31 ne répondaient pas à ces exigences. Ils ont été retirés du marché.</p>
<p><!-- scald=30345215:full {"context":"full","view_mode":"enacarbon_teaser_niveau_2_cipiz","scald_mediacontext":"full","dpiservices":false,"digiteka_dmp_targeting":1} --> <r-embed class="r-embed--digiteka"> <template class="dmp-digiteka-4ef4174b4883ba38b7bbf3eda12ce1d4"> <iframe allowfullscreen="true" class="lazy" data-digiteka-sid="30345215" data-src="https://www.ultimedia.com/deliver/generic/iframe/mdtk/01613302/zone/34/showtitle/1/src/qkvumrz/?tagparamdecoded=Business&amp;tagparam=cat%3Dbusiness" frameborder="0" height="400" hspace="0" marginheight="0" marginwidth="0" mozallowfullscreen="true" referrerpolicy="no-refe

In [34]:
def find_article_title(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    article_title = soup.find("h1").text
    return article_title

In [35]:
def find_article_text(url: str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    article_text = "".join([p.text.strip('\n') for p in soup.find_all('p')])
    return article_text

In [36]:
def find_published_date(url:str) -> str:
    response = requests.get(url)
    soup = bs(response.content, "html.parser")
    script = soup.find('script', {"type": "application/ld+json"})
    data = json.loads(script.text, strict=False)
    try:
        published_date = data['@graph'][0]['datePublished']
    except:
        published_date = data['datePublished']    
    return published_date

In [37]:
df['article_title'] = df['source_url'].apply(find_article_title)

In [38]:
df['article_text'] = df['source_url'].apply(find_article_text)

In [39]:
df['date'] = df['source_url'].apply(find_published_date)

In [40]:
df['language'] = 'fr'

In [42]:
df.head()

,source_url,article_title,article_text,date,language
0,https://www.lesoir.be/529026/article/2023-08-0...,Vol avec violences : la police lance un avis d...,Le samedi 15 juillet 2023 entre 18h40 et 19h00...,2023-08-02T15:57:51+02:00,fr
1,https://www.lesoir.be/529025/article/2023-08-0...,Près d’une tonne de cocaïne saisie et cinq arr...,Cinq personnes ont été arrêtées lundi dans un ...,2023-08-02T15:48:50+02:00,fr
2,https://www.lesoir.be/529024/article/2023-08-0...,Explosion du taux de criminalité au Japon : l’...,"Correspondant à Tokyo.Lundi, le tribunal de To...",2023-08-02T15:48:36+02:00,fr
3,https://www.lesoir.be/529022/article/2023-08-0...,"« Averses intenses », « rafales de vent », « o...","Ce mercredi 2 août et jeudi 3 août, l’IRM décl...",2023-08-02T15:45:38+02:00,fr
4,https://www.lesoir.be/529018/article/2023-08-0...,Guerre en Ukraine : les frappes russes sur le ...,Les frappes russes qui ont visé mercredi matin...,2023-08-02T15:32:27+02:00,fr


In [43]:
df.to_csv('data.lesoir_articles.csv')